In [12]:
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup as BSHTML
import bs4
import csv
import requests
import os

from tqdm.notebook import tqdm

In [13]:
os.listdir('data2')

['skoinewsHD',
 'quoteV2',
 'daghee',
 'JeebNuNoi',
 '.ipynb_checkpoints',
 'redskullnews',
 'The.RPG.Official',
 'ejeab',
 '1535671279901711',
 'DramaAdd',
 'Deakwans',
 'khaopodnews',
 'wanznewsHD',
 'humanlv.1']

In [14]:
features = ['page_id','thread_level', 'parent_id','post_id','post_text','post_text_raw', 'post_image', 'post_video','post_comment_count','post_share_count','post_reaction_count','post_like','post_love','post_haha','post_sorry','post_wow','post_anger','post_utime','comment_text','comment_text_raw','comment_id','comment_author','comment_author_id','comment_image','comment_video','comment_reaction_count','comment_reaction1','comment_reaction2','comment_reaction3','comment_topfan','comment_reply_count','comment_utime']
all_df = pd.DataFrame(columns=features)
dfs = []
num = 0

In [15]:
def append_to_df(d):
    global all_df
    global page
    global num
    all_df = all_df.append(pd.Series(d), ignore_index = True) 
    if len(all_df) % 1000 == 0:
        print(str(num) + " " + str(len(all_df)))
    if len(all_df) >= 50000:
        all_df.to_csv('processed_data/' + page + '_' + str(num) + '.csv')
        all_df = pd.DataFrame(columns=features)
        num = num + 1

def read_html(path):
    try:
        f = open(path, "r", encoding='utf-8')
        s = f.read()
        f.close()
        return s
    except OSError as e:
        return None
def get_post_script(s,postid):
    raw_script = re.findall('<script>(.+?)</script>',s)
    script = [i for i in raw_script if postid in str(i)]
    return script

def get_page_id(s):
    page_id = re.search('(?<=page_id.)[\d]+',str(s))
    if page_id == None:
        return ('<UNK>')
    else:
        return page_id.group()
    
def get_post_id(s):
    postid = s[6:s.index('.html')]
    return postid

def get_post_text(s):
    content = soup.find('div', class_= 'userContent', attrs={"data-testid":"post_message"})
    if content == None:
        return ('<NONE>')
    else:
        return content.get_text()

def get_post_text_raw(s):
    content = soup.find('div', class_= 'userContent', attrs={"data-testid":"post_message"})
    if content == None:
        return ('<NONE>')
    else:
        return content
    
def get_post_video(s):
    return 1 if ('permalink_video_pagelet' in s or f'{page}/video' in s) else 0

def get_post_image(s):
    return 1 if 'uiScaledImageContainer _517g' in s else 0

def count_comment():
    count = soup.find('a', class_ = '_3hg- _42ft')
    if count == None: 
        return (0)
    else:
        countt = count.get_text()
        count_comment = re.search(r'\d+', countt)
    return int(count_comment.group())

def count_share():
    count = soup.find('a', class_ = '_3rwx _42ft')
    if count == None:
        return (0)
    else:
        countt = count.get_text()
        count_share = re.search(r'\d+',countt)
    return int(count_share.group())

def post_all_reaction(s):
    reaction = re.search('(?<=reactors:{count:)[\d]+',str(s))   
    if reaction == None:
        return('Cannot count all reaction')
    else:
        return int(reaction.group())

def get_post_like(s):
    like_count = re.search('[\d]+(?=,node:{key:1)',str(s))
    if like_count == None:
        return (0)
    else:
        return int(like_count.group())

def get_post_love(s):
    love_count = re.search('[\d]+(?=,node:{key:2)',str(s))
    if love_count == None:
        return (0)
    else:
        return int(love_count.group())
    
def get_post_haha(s):
    haha_count = re.search('[\d]+(?=,node:{key:4)',str(s))
    if haha_count == None:
        return (0)
    else:
        return int(haha_count.group())
    
def get_post_wow(s):
    wow_count = re.search('[\d]+(?=,node:{key:3)',str(s))
    if wow_count == None:
        return (0)
    else:
        return int(wow_count.group())
    
def get_post_sorry(s):
    sorry_count = re.search('[\d]+(?=,node:{key:7)',str(s))
    if sorry_count == None:
        return (0)
    else:
        return int(sorry_count.group())

def get_post_anger(s):
    anger_count = re.search('[\d]+(?=,node:{key:8)',str(s))
    if anger_count == None:
        return (0)
    else:
        return int(anger_count.group())

def get_utime(post):
    time = re.search(r'(?<=data-utime=\")(\d+)', str(post))
    return int(time.group())

def get_time(post):
    time = re.search(r'(?<= )(\d+):(\d+)', str(post))
    return time.group()

def get_comment_text(comment):
    commenttext = comment.find('span', class_= "_3l3x")
    if commenttext == None:
        return (' ')
    else:
        return commenttext.text
    
def get_comment_text_raw(comment):
    commenttext = comment.find('span', class_= "_3l3x")
#     print (commenttext)
    if commenttext == None:
        return (' ')
    else:
        return commenttext

def get_comment_id(comment):
    commentid = re.findall('comment_id=(\d+">)',str(comment))
    if commentid == None:
        return( None)
    else:
        s = commentid[0]
        return (s[:-2])
    
def get_comment_author(comment):
    author = comment.find('a', class_ = "_6qw4")
    if author == None:
        author_eiei = comment.find('span', class_= "_6qw4")
        return author_eiei.get_text()
    else:
        return author.get_text()

def get_comment_author_id(comment):
#     print(comment)
    author_id = re.search('(?<=id=)(\d+)(?=&amp)', str(comment))
    if author_id == None:
        return ('User close an account')
    else:
        return author_id.group()

def get_comment_video(comment):
    comment_vid = comment.find('i', rel="presentation")
    if comment_vid == None:
        return (0)
    else:
        return (1)
    
def get_comment_picture(comment):
    comment_pic = comment.find('a', rel="theater")
    if comment_pic == None:
        return (0)
    else:
        return (1)
    
def count_comment_reaction(comment):
    commentreaction = comment.find('span',class_ ="_1lld")
    if commentreaction == None:
        return (0)
    else:
        return commentreaction.get_text()

def get_comment_reaction_123(comment):
    s = comment.find(attrs = {'class':'_1lh7'})
    reaction = re.findall('(?<=_3j7)[\w+]',str(s))
    reaction_dict = {'l':'like','m':'love','n':'wow','r':'sad','o':'haha','q':'angry'}

    if len(reaction) == 1:
        reaction1 = reaction_dict.get(reaction[0])
        reaction2,reaction3 = '',''
        return reaction1,reaction2,reaction3

    if len(reaction) == 2:
        reaction1 = reaction_dict.get(reaction[0])
        reaction2 = reaction_dict.get(reaction[1])
        reaction3 = ''
        return reaction1,reaction2,reaction3

    if len(reaction) == 3:
        reaction1 = reaction_dict.get(reaction[0])
        reaction2 = reaction_dict.get(reaction[1])
        reaction3 = reaction_dict.get(reaction[2])  
        return reaction1,reaction2,reaction3

    else:
        reaction1,reaction2,reaction3 = '','',''
        return reaction1,reaction2,reaction3

def get_comment_topfan(comment):
    fan = comment.find('span', class_= "_3x6z")
    if fan==None:
        return (0)
    else:
        return (1)

def count_sup_comment(comment):
    reply = comment.find('span', class_ = "_3_qo")
#     print(reply)
    reply_c = re.search(r'-?\d+\.?\d', str(reply))
    if reply == None:
        return (0)
    else:
        return reply_c.group()

def get_all_comment(comments, thread_level, parent_id, tmp_post_rec):
    global err_id_list
    for comment in comments:
        try:
            comment_id = get_comment_id(comment)
            current_id = comment_id
            comment_author = get_comment_author(comment)
            comment_author_id = get_comment_author_id(comment)
            comment_text = get_comment_text(comment)
            comment_text_raw = get_comment_text_raw(comment)
            comment_image = get_comment_picture(comment)
            comment_video = get_comment_video(comment)
            comment_reaction_count = count_comment_reaction(comment)
            comment_reaction_type = list(get_comment_reaction_123(comment))
            comment_reaction1 = comment_reaction_type[0]
            comment_reaction2 = comment_reaction_type[1]
            comment_reaction3 = comment_reaction_type[2]
            comment_topfan = get_comment_topfan(comment)
            comment_reply_count = count_sup_comment(comment)
            comment_utime = get_utime(comment)
            tmp_comment_rec = tmp_post_rec.copy()
            modifier = [('thread_level',thread_level), 
                ('parent_id', parent_id), 
                ('comment_text', comment_text),
                ('comment_text_raw', comment_text_raw), 
                ('comment_id', comment_id),
                ('comment_author', comment_author),
                ('comment_author_id', comment_author_id),
                ('comment_image', comment_image),
                ('comment_video', comment_video),
                ('comment_reaction_count', comment_reaction_count),
                ('comment_reaction1', comment_reaction1),
                ('comment_reaction2', comment_reaction2),
                ('comment_reaction3', comment_reaction3),
                ('comment_topfan', comment_topfan),
                ('comment_reply_count', comment_reply_count),
                ('comment_utime', comment_utime)]
            for i,v in modifier:
                tmp_comment_rec[i] = v
            append_to_df(tmp_comment_rec)
    #         print(tmp_comment_rec)
            sup = comment.findChildren('div', recursive = False)
    #         print(sup)
            if len(sup) <= 1:
                continue
            sup = sup[1]
            sup = sup.find('ul')
            if sup == None:
                continue
            else:
                sups = sup.findChildren('li', recursive = False)
                get_all_comment(sups, thread_level + 1, comment_id, tmp_post_rec)
        except:
            err_id_list.append(current_id)
            pass

In [16]:
page = 'humanlv.1'
data = []

# Features Description

'page_id': page id
'post_id': post id
'post_text': text in post
'post_text_raw': text without clean in post
'post_video': is this post has video (1: yes, 0: no)
'post_image': is this post has image (1: yes, 0: no)
'post_comment_count': number of comment in a post
'post_share_count': number of share in a post
'post_reaction_count': number of all reaction of post
'post_like',
'post_love',
'post_haha',
'post_sorry',
'post_wow',
'post_anger',
'post_utime': post time in u format
'comment_post_id',
'comment_text',
'comment_text_raw',
'comment_id',
'comment_author',
'comment_author_id',
'comment_image',
'comment_video',
'comment_reaction_count',
'comment_reaction1',
'comment_reaction2',
'comment_reaction3',
'comment_topfan',
'comment_reply_count',
'comment_utime',
'parent_id': can be both post id and another comment id
'thread_level': 0: post, 1: comment, 2: reply to comment

In [17]:
posts = os.listdir(f'data2/{page}')
posts = [i for i in posts if 'postId' in i and '.html' in i]

In [18]:
# posts = posts[:5]

In [19]:
current_id = 0

In [20]:
err_id_list = []

In [21]:
for post in tqdm(posts, desc = 'post'):
    try:
        s = read_html(f'data2/{page}/{post}')
        soup = bs4.BeautifulSoup(s,'lxml')
        post_id = get_post_id(post)
        script = get_post_script(s, post_id)
        page_id = get_page_id(script)
        current_id = post_id
        post_text = get_post_text(s)
        post_text = post_text.replace(',', '')
        post_text_raw = str(get_post_text_raw(s))
        post_text_raw = post_text_raw.replace(',', '')
        post_image = get_post_image(s)
        post_video = get_post_video(s)
        post_comment_count = count_comment()
        post_share_count = count_share()
        post_reaction_count = post_all_reaction(script)
        post_like = get_post_like(script)
        post_love = get_post_love(script)
        post_haha = get_post_haha(script)
        post_wow = get_post_wow(script)
        post_sorry = get_post_sorry(script)
        post_anger = get_post_anger(script)
        post_utime = get_utime(s)
        tmp_post_rec = {'page_id': page_id,
                        'thread_level':0,
                        'parent_id': page_id,
                        'post_id': post_id,
                        'post_text': post_text,
                        'post_text_raw': post_text_raw,
                        'post_image': post_image,
                        'post_video': post_video,
                        'post_comment_count': post_comment_count,
                        'post_share_count': post_share_count,
                        'post_reaction_count': post_reaction_count,
                        'post_like': post_like,
                        'post_love': post_love,
                        'post_haha': post_haha,
                        'post_sorry': post_sorry,
                        'post_wow': post_wow,
                        'post_anger': post_anger,
                        'post_utime': post_utime}
        append_to_df(tmp_post_rec)
        comments = soup.find_all('ul', class_ = '_7a9a')
        if  len(comments) == 0:
            print('this post has no comment')
        comments = list(comments[0].children)
        get_all_comment(comments, 1, post_id, tmp_post_rec)
    except:
        err_id_list.append(current_id)

all_df.to_csv('processed_data/' + page + '_' + str(num) + '.csv')

0 1000
0 2000
0 3000
this post has no comment
0 4000
0 5000
0 6000
0 7000
0 8000
0 9000
0 10000
0 11000
0 12000
0 13000
0 14000
0 15000
0 16000
0 17000
0 18000
0 19000
0 20000
0 21000
0 22000
0 23000
0 24000
0 25000
0 26000
0 27000
0 28000
0 29000
0 30000
0 31000



# End